<a href="https://colab.research.google.com/github/hallosayaimroatubelajargithub/Sistem-Rekomendasi-Pembelajaran/blob/main/content_based_filtering_pembelajaran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Content Based Filtering : Rekomendasi Modul Pembelajaran**

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

df = pd.read_csv("https://raw.githubusercontent.com/hallosayaimroatubelajargithub/Sistem-Rekomendasi-Pembelajaran/main/data_rekomendasi.csv")
df.head()

,mitra,alamat,modul
0,PT Orbit Ventura Indonesia,"Veteran RI Building 15th Floor Unit\nZ15-002, ...",Project AI Metode Penelitian AI Pemrograma...
1,PT Nurul Fikri Cipta Inovasi,Jl. Situ Indah No.116 RT. 006 \nRW. 010 Kel. T...,Desain Web Git/Github Javascript Laravel...
2,PT MariBelajar Indonesia \nCerdas,"Jln Tunggul Ametung IVa no 8, \nDesa Ubung Kaj...",Microsoft 365 Fundamentals Microsoft Azure F...
3,PT Mitra Integrasi Informatika,"APL Tower Lantai 37, Jl. Letjen \nS. Parman Ka...",Introduction Programming Front-End Programmi...
4,PT. Impactbyte Teknologi Edukasi,"Jl. Simprug Golf 8 No..6, RT.2/RW.8, \nGrogol ...",User Interface Designing dan Prototyping Res...


**1. Ikhtisar**

In [2]:
df.describe()

,mitra,alamat,modul
count,11,11,11
unique,10,11,11
top,PT Mitra Integrasi Informatika,"Veteran RI Building 15th Floor Unit\nZ15-002, ...",Project AI Metode Penelitian AI Pemrograma...
freq,2,1,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   mitra   11 non-null     object
 1   alamat  11 non-null     object
 2   modul   11 non-null     object
dtypes: object(3)
memory usage: 392.0+ bytes


**2. Deskripsi Modul Pembelajaran (Sebelum Preprocessing)**

In [4]:
def print_description(index):
    example = df[df.index == index][['modul', 'mitra','alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('mitra:', example[1])
        print('alamat:', example[2])

In [5]:
print_description(1)

Desain Web   Git/Github   Javascript   Laravel   Database MYSQL   PHP   REST API   UI/UX   Soft Skill   Final Project
mitra: PT Nurul Fikri Cipta Inovasi
alamat: Jl. Situ Indah No.116 RT. 006 
RW. 010 Kel. Tugu, Kec. Cimanggis, 
Kota Depok, Jawa Barat



In [6]:
print_description(3)

Introduction Programming   Front-End Programming   Back-End Programming   Secure Programming   
Power Platform Fundamental
mitra: PT Mitra Integrasi Informatika
alamat: APL Tower Lantai 37, Jl. Letjen 
S. Parman Kav. 28, RT.12/RW.6, 
Tj. Duren Sel., Jakarta Barat, 
Kota Jakarta Barat, 
Daerah Khusus Ibukota Jakarta 11470


In [19]:
print_description(5)

Text Mining   Machine Learning   Basis Data   Pemrograman Web   Jejaring Sosial   Web Service   Pemrograman Aplikasi Mobile   
Rekayasa Perangkat Lunak
mitra: Imroatu
alamat: Desel, Ngaliyan Semarang Jawa Tengah


**3. Text Preprocessing**

In [20]:
import nltk
nltk.download('stopwords')
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
#stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    #text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['modul_pembelajaran_clean'] = df['modul'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
df.head()

,mitra,alamat,modul,modul_pembelajaran_clean
0,PT Orbit Ventura Indonesia,"Veteran RI Building 15th Floor Unit\nZ15-002, ...",Project AI Metode Penelitian AI Pemrograma...,project ai metode penelitian ai pemrograma...
1,PT Nurul Fikri Cipta Inovasi,Jl. Situ Indah No.116 RT. 006 \nRW. 010 Kel. T...,Desain Web Git/Github Javascript Laravel...,desain web git github javascript laravel...
2,PT MariBelajar Indonesia \nCerdas,"Jln Tunggul Ametung IVa no 8, \nDesa Ubung Kaj...",Microsoft 365 Fundamentals Microsoft Azure F...,microsoft 365 fundamentals microsoft azure f...
3,PT Mitra Integrasi Informatika,"APL Tower Lantai 37, Jl. Letjen \nS. Parman Ka...",Introduction Programming Front-End Programmi...,introduction programming frontend programmin...
4,PT. Impactbyte Teknologi Edukasi,"Jl. Simprug Golf 8 No..6, RT.2/RW.8, \nGrogol ...",User Interface Designing dan Prototyping Res...,user interface designing dan prototyping res...


**4. Deskripsi Modul Pembelajaran (Setelah Preprocessing)**

In [22]:
# Deskripsi kedua (Setelah preprocessing)
def print_description_clean(index):
    example = df[df.index == index][['modul_pembelajaran_clean', 'mitra','alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('mitra:', example[1])
        print('alamat:', example[2])

In [23]:
print_description_clean(1)

desain web   git github   javascript   laravel   database mysql   php   rest api   ui ux   soft skill   final project
mitra: PT Nurul Fikri Cipta Inovasi
alamat: Jl. Situ Indah No.116 RT. 006 
RW. 010 Kel. Tugu, Kec. Cimanggis, 
Kota Depok, Jawa Barat



In [24]:
print_description_clean(2)

microsoft 365 fundamentals   microsoft azure fundamentals   capstone projetct   design thingking masterclass   modern project management   microsoft azure administrator
mitra: PT MariBelajar Indonesia 
Cerdas
alamat: Jln Tunggul Ametung IVa no 8, 
Desa Ubung Kaja, Kecamatan 
Denpasar utara, Kota Denpasar


In [25]:
print_description_clean(5)

text mining   machine learning   basis data   pemrograman web   jejaring sosial   web service   pemrograman aplikasi mobile   rekayasa perangkat lunak
mitra: Imroatu
alamat: Desel, Ngaliyan Semarang Jawa Tengah


**5. TF-IDF & Cosine Similarity**

In [26]:
df.set_index('mitra', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['modul_pembelajaran_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.01252537, 0.01114433, 0.        , 0.        ,
        0.0303574 , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.01252537, 1.        , 0.01094202, 0.        , 0.05123811,
        0.02980632, 0.        , 0.        , 0.        , 0.        ,
        0.06367502],
       [0.01114433, 0.01094202, 1.        , 0.        , 0.03977898,
        0.        , 0.02577011, 0.        , 0.        , 0.03822683,
        0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.05123811, 0.03977898, 0.        , 1.        ,
        0.        , 0.0484071 , 0.        , 0.05810861, 0.05135387,
        0.        ],
       [0.0303574 , 0.02980632, 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.01990878, 0.020046  ,
        0.        ],
       [0.        , 0.        , 0.02577011, 0.        , 0.

In [27]:
# Set index utama di kolom 'name'
indices = pd.Series(df.index)
indices[:50]

0                PT Orbit Ventura Indonesia
1              PT Nurul Fikri Cipta Inovasi
2         PT MariBelajar Indonesia \nCerdas
3            PT Mitra Integrasi Informatika
4          PT. Impactbyte Teknologi Edukasi
5                                   Imroatu
6     PT. IlmuKomputerCom Braindevs Sistema
7            PT Mitra Integrasi Informatika
8              PT Semesta Integrasi Digital
9         PT Telkom Indonesia (Persero) Tbk
10      PT Abbauf Mulia Konsultan Teknologi
Name: mitra, dtype: object

**6. Modelling**

In [28]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_hotel = []
    
    # Mengambil nama hotel berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 10 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:4].index)
    
    for i in top_10_indexes:
        recommended_hotel.append(list(df.index)[i])
        
    return recommended_hotel

**7. Prediksi**

In [29]:
recommendations('Imroatu')

['PT Orbit Ventura Indonesia',
 'PT Nurul Fikri Cipta Inovasi',
 'PT Telkom Indonesia (Persero) Tbk']

In [30]:
recommendations('PT Telkom Indonesia (Persero) Tbk')

['PT. IlmuKomputerCom Braindevs Sistema',
 'PT. Impactbyte Teknologi Edukasi',
 'PT Semesta Integrasi Digital']

In [31]:
recommendations('PT Nurul Fikri Cipta Inovasi')

['PT Abbauf Mulia Konsultan Teknologi',
 'PT. Impactbyte Teknologi Edukasi',
 'Imroatu']